In [3]:
import requests
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import os
import subprocess
import tempfile
import sys
import time
from bs4 import BeautifulSoup
import re

class EnhancedRAGSystem:
    def __init__(self, knowledge_base_path="knowledge_base.pkl", search_enabled=True):
        self.knowledge_base_path = knowledge_base_path
        self.knowledge_base = []
        self.embeddings = []
        self.search_enabled = search_enabled
        
        self._initialize_knowledge_base()
    
    def _initialize_knowledge_base(self):
        """Инициализирует базу знаний с информацией о numpy и tensorflow"""
        if os.path.exists(self.knowledge_base_path):
            self._load_knowledge_base()
        else:
            self._create_knowledge_base()
            self._save_knowledge_base()
    
    def _create_knowledge_base(self):
        """Создает базовую базу знаний"""
        self.knowledge_base = [
            {
                "content": "NumPy - Python library for scientific computing. Main object - ndarray. Creation: np.array([1,2,3]), np.zeros((3,3)), np.ones((2,2)), np.arange(10), np.linspace(0, 1, 5), np.random.rand(3,3)",
                "source": "numpy_basics",
                "keywords": ["numpy", "array", "scientific computing", "creation"]
            },
            {
                "content": "TensorFlow - open source ML framework. Key concepts: Tensors (multidimensional arrays), Operations (tf.add, tf.matmul), Graphs (computational graphs), Sessions (tf.Session). Latest version: TensorFlow 2.x with eager execution enabled by default.",
                "source": "tensorflow_basics", 
                "keywords": ["tensorflow", "tensors", "machine learning", "operations"]
            },
            {
                "content": "NumPy array operations: np.dot() for dot product, np.transpose() for transpose, np.reshape() to change shape, np.concatenate() to join arrays, np.split() to split arrays, np.sum() for summation along axis.",
                "source": "numpy_operations",
                "keywords": ["numpy", "operations", "dot product", "reshape", "sum"]
            },
            {
                "content": "TensorFlow 2.x uses Keras as high-level API. Key modules: tf.keras.layers (Dense, Conv2D, LSTM), tf.keras.models (Sequential, Model), tf.keras.optimizers (Adam, SGD), tf.keras.losses (categorical_crossentropy, mse).",
                "source": "tensorflow_keras",
                "keywords": ["tensorflow", "keras", "layers", "models", "optimizers"]
            },
            {
                "content": "NumPy indexing and slicing: arr[0] for first element, arr[1:5] for slice, arr[arr > 5] for boolean indexing, arr[:, 0] for column access, fancy indexing with arr[[1,3,5]]. Copy vs view: arr.copy() for explicit copy.",
                "source": "numpy_indexing",
                "keywords": ["numpy", "indexing", "slicing", "boolean indexing"]
            },
            {
                "content": "TensorFlow datasets: tf.data.Dataset for efficient data pipeline. Methods: .from_tensor_slices(), .batch(), .shuffle(), .map(), .prefetch(). Use for training loops with for batch in dataset.",
                "source": "tensorflow_data",
                "keywords": ["tensorflow", "dataset", "data pipeline", "training"]
            }
        ]
        # Создаем простые эмбеддинги на основе ключевых слов
        self.embeddings = self._create_simple_embeddings()
    
    def _create_simple_embeddings(self):
        """Создает простые эмбеддинги на основе ключевых слов"""
        embeddings = []
        for doc in self.knowledge_base:
            # Простой one-hot like encoding на основе ключевых слов
            embedding = np.zeros(100)  # Фиксированный размер
            keywords = doc["keywords"]
            
            # Хэшируем ключевые слова в индексы
            for keyword in keywords:
                idx = hash(keyword) % 100
                embedding[idx] = 1.0
            
            # Добавляем немного случайности для разнообразия
            embedding += np.random.normal(0, 0.1, 100)
            embeddings.append(embedding)
        
        return np.array(embeddings)
    
    def _save_knowledge_base(self):
        """Сохраняет базу знаний"""
        with open(self.knowledge_base_path, 'wb') as f:
            pickle.dump({
                'knowledge_base': self.knowledge_base,
                'embeddings': self.embeddings
            }, f)
        print(f"💾 Knowledge base saved with {len(self.knowledge_base)} documents")
    
    def _load_knowledge_base(self):
        """Загружает базу знаний"""
        with open(self.knowledge_base_path, 'rb') as f:
            data = pickle.load(f)
            self.knowledge_base = data['knowledge_base']
            self.embeddings = data['embeddings']
        print(f"📂 Knowledge base loaded with {len(self.knowledge_base)} documents")
    
    def _search_online(self, query, max_results=3):
        """Ищет информацию в Интернете"""
        if not self.search_enabled:
            return []
            
        print(f"🌐 Searching online for: {query}")
        
        try:
            # Поиск через DuckDuckGo или другой поисковик
            search_url = "https://api.duckduckgo.com/"
            params = {
                "q": f"{query} numpy tensorflow python",
                "format": "json",
                "no_html": 1,
                "skip_disambig": 1
            }
            
            response = requests.get(search_url, params=params, timeout=10)
            if response.status_code == 200:
                data = response.json()
                results = []
                
                # Извлекаем Abstract (краткое описание)
                if data.get('Abstract'):
                    results.append({
                        'content': data['Abstract'],
                        'source': 'duckduckgo_abstract',
                        'keywords': query.lower().split()
                    })
                
                # Извлекаем RelatedTopics
                for topic in data.get('RelatedTopics', [])[:max_results]:
                    if 'Text' in topic:
                        results.append({
                            'content': topic['Text'],
                            'source': 'duckduckgo_related',
                            'keywords': query.lower().split()
                        })
                
                return results
                
        except Exception as e:
            print(f"⚠️ Online search failed: {e}")
        
        return []
    
    def _scrape_webpage(self, url):
        """Сканирует веб-страницу для извлечения контента"""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Удаляем скрипты и стили
            for script in soup(["script", "style"]):
                script.decompose()
            
            # Извлекаем текст
            text = soup.get_text()
            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            text = ' '.join(chunk for chunk in chunks if chunk)
            
            return text[:1000]  # Ограничиваем длину
            
        except Exception as e:
            print(f"⚠️ Web scraping failed: {e}")
            return ""
    
    def _text_to_embedding(self, text):
        """Преобразует текст в эмбеддинг"""
        words = text.lower().split()
        embedding = np.zeros(100)
        
        for word in words:
            if len(word) > 2:  # Игнорируем очень короткие слова
                idx = hash(word) % 100
                embedding[idx] += 1.0
        
        # Нормализуем
        if np.linalg.norm(embedding) > 0:
            embedding = embedding / np.linalg.norm(embedding)
        
        return embedding
    
    def search(self, query, top_k=3, use_online=True):
        """Ищет релевантные документы, при необходимости ищет онлайн"""
        query_embedding = self._text_to_embedding(query)
        
        # Вычисляем косинусное сходство для локальных документов
        similarities = []
        for doc_embedding in self.embeddings:
            similarity = np.dot(query_embedding, doc_embedding) / (
                np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding) + 1e-8
            )
            similarities.append(similarity)
        
        # Получаем топ-K результатов
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        
        results = []
        online_results_added = False
        
        for idx in top_indices:
            if similarities[idx] > 0.1:  # Минимальный порог сходства
                results.append({
                    'content': self.knowledge_base[idx]['content'],
                    'source': self.knowledge_base[idx]['source'],
                    'score': similarities[idx],
                    'type': 'local'
                })
        
        # Если локальных результатов недостаточно или они нерелевантны, ищем онлайн
        if use_online and self.search_enabled and (not results or max(similarities) < 0.4):
            online_results = self._search_online(query, max_results=top_k)
            
            for online_doc in online_results:
                # Создаем эмбеддинг для онлайн документа
                online_embedding = self._text_to_embedding(online_doc['content'])
                
                # Вычисляем сходство с запросом
                online_similarity = np.dot(query_embedding, online_embedding) / (
                    np.linalg.norm(query_embedding) * np.linalg.norm(online_embedding) + 1e-8
                )
                
                if online_similarity > 0.2:  # Порог для онлайн результатов
                    results.append({
                        'content': online_doc['content'],
                        'source': online_doc['source'],
                        'score': online_similarity,
                        'type': 'online'
                    })
                    
                    # Добавляем в локальную базу знаний
                    self._add_to_knowledge_base(online_doc['content'], online_doc['source'], query.lower().split())
                    online_results_added = True
            
            # Сохраняем базу знаний, если были добавлены новые документы
            if online_results_added:
                self._save_knowledge_base()
        
        return results if results else [{
            'content': 'No relevant documentation found locally or online', 
            'source': 'none', 
            'score': 0,
            'type': 'none'
        }]
    
    def _add_to_knowledge_base(self, content, source, keywords):
        """Добавляет новый документ в базу знаний"""
        # Проверяем, нет ли уже похожего документа
        new_embedding = self._text_to_embedding(content)
        
        for existing_embedding in self.embeddings:
            similarity = np.dot(new_embedding, existing_embedding) / (
                np.linalg.norm(new_embedding) * np.linalg.norm(existing_embedding) + 1e-8
            )
            if similarity > 0.8:  # Высокий порог сходства - вероятно дубликат
                print("📝 Similar document already exists, skipping...")
                return
        
        # Добавляем новый документ
        new_doc = {
            "content": content,
            "source": f"online_{source}_{int(time.time())}",
            "keywords": keywords
        }
        
        self.knowledge_base.append(new_doc)
        self.embeddings = np.vstack([self.embeddings, new_embedding])
        
        print(f"📚 Added new document to knowledge base from {source}")

class CodeCritic:
    """Агент для критики и анализа кода"""
    
    def __init__(self, rag_system):
        self.rag_system = rag_system
    
    def analyze_code(self, code, original_query):
        """Анализирует код и предоставляет критические замечания"""
        # Поиск релевантной документации для анализа кода
        critique_docs = self.rag_system.search("code quality best practices tensorflow numpy", top_k=3)
        
        critique_prompt = f"""
        Analyze the following code generated in response to the query: "{original_query}"
        
        Code to analyze:
        ```python
        {code}
        ```
        
        Provide a critical analysis focusing on:
        1. Correctness and functionality
        2. Code quality and best practices
        3. Potential bugs or issues
        4. Performance considerations
        5. Suggestions for improvement
        
        Reference documentation:
        {''.join([f"- {doc['content']}\\n" for doc in critique_docs])}
        
        Be constructive and specific in your criticism.
        """
        
        return self._get_llm_response(critique_prompt, "code_critic"), critique_docs
    
    def _get_llm_response(self, prompt, role="code_critic"):
        """Получает ответ от LLM"""
        url = "http://127.0.0.1:8080/v1/chat/completions"
        
        data = {
            "model": "unsloth/Llama-3.2-1B-Instruct-GGUF",
            "messages": [
                {
                    "role": "system", 
                    "content": "You are an expert code reviewer and software engineer. Provide detailed, constructive criticism of code quality, correctness, and best practices. Focus on making the code as simple as possible in each iteration. Fix the code rather than adding unnecessary entities; on the contrary, eliminate unnecessary entities by making the code simpler."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            "stream": False
        }
        
        try:
            response = requests.post(url, json=data)
            if response.status_code == 200:
                return response.json()['choices'][0]['message']['content']
            else:
                return f"Error: Unable to get response from LLM. Status code: {response.status_code}"
        except Exception as e:
            return f"Error: {e}"

class CodeExecutor:
    """Агент для выполнения и тестирования кода"""
    
    def __init__(self):
        self.error_history = {}  # Словарь для отслеживания ошибок по запросам
    
    def execute_code(self, code, query):
        """Выполняет код и возвращает результат или ошибку"""
        # Создаем временный файл для выполнения кода
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            # Добавляем необходимые импорты
            full_code = "import numpy as np\nimport tensorflow as tf\n\n" + code
            f.write(full_code)
            temp_file = f.name
        
        try:
            # Выполняем код
            result = subprocess.run(
                [sys.executable, temp_file],
                capture_output=True,
                text=True,
                timeout=30  # 30 секунд таймаут
            )
            
            # Удаляем временный файл
            os.unlink(temp_file)
            
            if result.returncode == 0:
                return {
                    'success': True,
                    'output': result.stdout,
                    'error': None
                }
            else:
                error_msg = result.stderr
                self._record_error(query, error_msg)
                return {
                    'success': False,
                    'output': None,
                    'error': error_msg
                }
                
        except subprocess.TimeoutExpired:
            os.unlink(temp_file)
            error_msg = "Execution timeout exceeded (30 seconds)"
            self._record_error(query, error_msg)
            return {
                'success': False,
                'output': None,
                'error': error_msg
            }
        except Exception as e:
            if os.path.exists(temp_file):
                os.unlink(temp_file)
            error_msg = f"Execution failed: {str(e)}"
            self._record_error(query, error_msg)
            return {
                'success': False,
                'output': None,
                'error': error_msg
            }
    
    def _record_error(self, query, error_msg):
        """Записывает ошибку в историю"""
        if query not in self.error_history:
            self.error_history[query] = []
        
        # Сохраняем только последние 5 ошибок для каждого запроса
        self.error_history[query].append(error_msg)
        if len(self.error_history[query]) > 5:
            self.error_history[query] = self.error_history[query][-5:]
    
    def should_regenerate_from_scratch(self, query):
        """Проверяет, нужно ли перегенерировать код с нуля"""
        if query not in self.error_history:
            return False
        
        errors = self.error_history[query]
        if len(errors) < 2:
            return False
        
        # Проверяем, повторяется ли последняя ошибка
        last_error = errors[-1]
        previous_errors = errors[:-1]
        
        # Считаем, что ошибка повторяется, если похожая ошибка встречалась ранее
        similar_errors = 0
        for error in previous_errors:
            if self._are_errors_similar(last_error, error):
                similar_errors += 1
        
        return similar_errors >= 1  # Если хотя бы одна похожая ошибка была ранее
    
    def _are_errors_similar(self, error1, error2):
        """Проверяет, похожи ли две ошибки"""
        # Простая эвристика для сравнения ошибок
        error1_lower = error1.lower()
        error2_lower = error2.lower()
        
        # Общие ключевые слова для похожих ошибок
        common_patterns = [
            'nameerror', 'attributeerror', 'typeerror', 
            'valueerror', 'importerror', 'syntaxerror',
            'undefined', 'not defined', 'no module',
            'cannot import', 'module not found'
        ]
        
        # Если обе ошибки содержат одинаковые ключевые паттерны
        for pattern in common_patterns:
            if pattern in error1_lower and pattern in error2_lower:
                return True
        
        # Если ошибки достаточно похожи по содержанию
        words1 = set(error1_lower.split())
        words2 = set(error2_lower.split())
        common_words = words1.intersection(words2)
        
        similarity = len(common_words) / max(len(words1), len(words2))
        return similarity > 0.3

def enhance_prompt_with_rag(user_query, rag_system):
    """Улучшает промпт с помощью RAG и возвращает как улучшенный промпт, так и документы"""
    relevant_docs = rag_system.search(user_query, top_k=3)
    
    enhanced_prompt = f"{user_query}\n\nRelevant documentation:\n"
    
    for i, doc in enumerate(relevant_docs, 1):
        source_type = "🌐 ONLINE" if doc['type'] == 'online' else "💾 LOCAL"
        enhanced_prompt += f"- [{source_type}] {doc['content']}\n"
    
    return enhanced_prompt, relevant_docs

def get_llm_response(prompt, rag_system, role="assistant"):
    """Получает ответ от LLM с улучшенным промптом"""
    enhanced_prompt, rag_docs = enhance_prompt_with_rag(prompt, rag_system)
    
    url = "http://127.0.0.1:8080/v1/chat/completions"
    
    data = {
        "model": "unsloth/Llama-3.2-1B-Instruct-GGUF",
        "messages": [
            {
                "role": "system", 
                "content": "You are a helpful AI assistant. Use the provided reference information about numpy and tensorflow to write accurate and correct code examples."
            },
            {
                "role": "user", 
                "content": enhanced_prompt
            }
        ],
        "stream": False
    }
    
    try:
        response = requests.post(url, json=data)
        if response.status_code == 200:
            return response.json()['choices'][0]['message']['content'], rag_docs
        else:
            return f"Error: Unable to get response from LLM. Status code: {response.status_code}", []
    except Exception as e:
        return f"Error: {e}", []

def extract_code_from_response(response):
    """Извлекает код из ответа LLM"""
    if "```python" in response:
        start = response.find("```python") + 9
        end = response.find("```", start)
        return response[start:end].strip()
    elif "```" in response:
        start = response.find("```") + 3
        end = response.find("```", start)
        return response[start:end].strip()
    else:
        return response

def display_rag_docs(docs, title="RAG Documentation"):
    """Отображает RAG документы в удобном формате"""
    print(f"\n📚 {title}:")
    print("-" * 60)
    for i, doc in enumerate(docs, 1):
        source_type = "🌐 ONLINE" if doc['type'] == 'online' else "💾 LOCAL"
        print(f"{i}. [{source_type}] [{doc['source']}] {doc['content'][:150]}...")
        print(f"   Score: {doc['score']:.3f}")
        print()

def main():
    # Инициализируем улучшенную RAG систему, критика и исполнителя кода
    rag_system = EnhancedRAGSystem(search_enabled=True)
    critic = CodeCritic(rag_system)
    executor = CodeExecutor()
    
    print("=== Enhanced Multi-Agent Code Generation with Online RAG ===")
    print("Type 'quit' to exit.\n")
    
    while True:
        # Получаем запрос от пользователя
        user_query = input("\nEnter your code generation request: ")
        if user_query.lower() == 'quit':
            break
        
        # Запрашиваем количество итераций
        try:
            iterations = int(input("How many improvement iterations? (default: 3): ") or "3")
        except ValueError:
            iterations = 3
        
        original_query = user_query
        current_code = ""
        regeneration_count = 0
        max_regenerations = 2  # Максимальное количество перегенераций с нуля
        
        for iteration in range(iterations):
            print(f"\n{'='*60}")
            print(f"ITERATION {iteration + 1}/{iterations}")
            if regeneration_count > 0:
                print(f"REGENERATION ATTEMPT {regeneration_count}/{max_regenerations}")
            print(f"{'='*60}")
            
            # Проверяем, нужно ли перегенерировать код с нуля
            if executor.should_regenerate_from_scratch(original_query) and regeneration_count < max_regenerations:
                print("\n🔄 EXECUTOR: Detected repeating errors, regenerating code from scratch...")
                regeneration_count += 1
                # Очищаем историю ошибок для этого запроса
                if original_query in executor.error_history:
                    executor.error_history[original_query] = []
            
            # Генерация кода
            print("\n🤖 GENERATOR: Generating code...")
            if iteration == 0 or regeneration_count > 0:
                code_response, rag_docs = get_llm_response(user_query, rag_system)
            else:
                # На последующих итерациях используем улучшенный запрос
                improved_query = f"Improve the following code based on the criticism. Original query: {original_query}\n\nPrevious code:\n```python\n{current_code}\n```\n\nCritique:\n{critique}"
                code_response, rag_docs = get_llm_response(improved_query, rag_system)
            
            # Отображаем RAG документы
            display_rag_docs(rag_docs, "RAG Documentation Used for Generation")
            
            generated_code = extract_code_from_response(code_response)
            current_code = generated_code
            
            print("Generated code:")
            print("```python")
            print(generated_code)
            print("```")
            
            # Тестирование кода
            print("\n⚡ EXECUTOR: Testing code execution...")
            execution_result = executor.execute_code(generated_code, original_query)
            
            if execution_result['success']:
                print("✅ Code executed successfully!")
                if execution_result['output']:
                    print(f"Output: {execution_result['output']}")
            else:
                print("❌ Code execution failed!")
                print(f"Error: {execution_result['error']}")
            
            # Анализ кода критиком (кроме последней итерации)
            if iteration < iterations - 1:
                print("\n🔍 CRITIC: Analyzing code...")
                critique, critique_docs = critic.analyze_code(generated_code, original_query)
                
                # Добавляем информацию об ошибке выполнения в критику, если есть
                if not execution_result['success']:
                    critique = f"EXECUTION ERROR: {execution_result['error']}\n\n" + critique
                
                # Отображаем RAG документы критика
                display_rag_docs(critique_docs, "RAG Documentation Used for Critique")
                
                print("Code analysis:")
                print(critique)
            else:
                print("\n✅ Final iteration completed!")
        
        print(f"\n🎉 Completed {iterations} iterations for: '{original_query}'")
        if regeneration_count > 0:
            print(f"🔄 Code was regenerated from scratch {regeneration_count} time(s) due to repeating errors")

if __name__ == "__main__":
    main()

📂 Knowledge base loaded with 75 documents
=== Enhanced Multi-Agent Code Generation with Online RAG ===
Type 'quit' to exit.




Enter your code generation request:  Write simple neural network using TensorFlow.
How many improvement iterations? (default: 3):  8



ITERATION 1/8

🤖 GENERATOR: Generating code...
🌐 Searching online for: Write simple neural network using TensorFlow.

📚 RAG Documentation Used for Generation:
------------------------------------------------------------
1. [💾 LOCAL] [numpy_attributes] Array attributes: ndarray.shape - array dimensions, ndarray.dtype - data type, ndarray.size - total elements, ndarray.ndim - number of dimensions, nda...
   Score: 0.321

2. [💾 LOCAL] [tensorflow_cluster] Cluster training: tf.distribute.cluster_resolver.TFConfigClusterResolver() - cluster configuration, tf.config.experimental_connect_to_cluster() - conn...
   Score: 0.297

3. [💾 LOCAL] [tensorflow_control_flow] Control flow: tf.cond() - conditional execution, tf.while_loop() - while loops, tf.switch_case() - switch statements, automatically handled in eager e...
   Score: 0.259

Generated code:
```python
# Import necessary libraries
import numpy as np
import tensorflow as tf

# Set the random seed for reproducibility
np.random.seed(0)
tf

KeyboardInterrupt: 